In [ ]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import matplotlib.pyplot as plt

import pandas_datareader.data as web


In [ ]:
# Подготовка датасета для обучения в формате inputs, outputs
def make_datasets(input_data, n_inputs=2, n_outputs=1, gap=0):
	L = len(input_data)
	y = np.full((L-n_inputs-n_outputs-gap, n_outputs), 0.0)
	X = np.full((L-n_inputs-n_outputs-gap, n_inputs), 0.0)

	for i in range(n_inputs):
		X[:,i] = input_data[i:L-n_inputs-n_outputs-gap+i]

	for i in range(n_outputs):
		y[:,i] = input_data[n_inputs+gap+i:L-n_outputs+i]

	return X, y

In [ ]:
rate = web.DataReader(name='WGS10YR', data_source='fred', start='2000-01-01')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Переведем ряда массив Numpy
series = rate.iloc[:,0].values

# scaler = MinMaxScaler()
# series = scaler.fit_predict(series)

In [ ]:
# задаём ширину окна и горизонт прогнозирования
n_lags, fh= 20, 5

X, y = make_datasets(series, n_inputs=n_lags, n_outputs=fh)

In [ ]:
X_tensor = torch.Tensor(X).to(device)
y_tensor = torch.Tensor(y).to(device)

In [ ]:
train_dataset = TensorDataset(X_tensor, y_tensor)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [ ]:
class ExtractTensor(nn.Module):
    def forward(self, x):
        # Если x - кортеж, берем первый элемент
        return x[0] if isinstance(x, tuple) else x

In [ ]:
model = nn.Sequential(
    nn.LSTM(n_lags, 50),
	ExtractTensor(),
	# nn.Linear(n_lags, 50),
    nn.ReLU(),
    nn.Linear(50, fh)
).to(device)

In [ ]:
# Обучение
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters())



for epoch in range(10):
	total_loss = 0.0
	model.train()

	for batch_X, batch_y in train_loader:
		batch_X, batch_y = batch_X.to(device), batch_y.to(device)
		predictions = model(batch_X)
		loss = criterion(predictions, batch_y)

		optimizer.zero_grad()
		loss.backward()
		optimizer.step()

		total_loss += loss.item()

	print(f"Эпоха {epoch+1}, Loss: {loss.item():.4f}")

In [ ]:
test_loss = 0
inputs = torch.Tensor(np.reshape(series[-n_lags:], (1, n_lags))).to(device)

model.eval()
with torch.no_grad():  # Отключаем вычисление градиентов
	# for batch in test_loader:
	outputs = model(inputs)
	# test_loss += criterion(outputs, targets)

In [ ]:
outputs.numpy()